In [ ]:
pip install tensorflow-gpu keras scikit-learn tqdm

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Путь к загружаемой модели
model_path = '/home/jupyter/datasphere/project/model.h5'

# Загрузка модели
model = load_model(model_path)
print(f"Модель успешно загружена из файла {model_path}")

# Загрузка нового файла с данными (например, в формате CSV)
new_data_path = '/home/jupyter/datasphere/project/TEST_RES.csv'  # Укажите путь к вашему новому файлу
new_df = pd.read_csv(new_data_path)

# Предполагается, что в новом файле есть столбцы 'achievements_modified' и 'demands'
# Создание текстовых данных для предсказания
new_text_data = []
for _, row in new_df.iterrows():
    combined_text = f"{row['achievements_modified']} {row['demands']}"
    new_text_data.append(combined_text)

# Токенизация новых текстов
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(new_text_data)  # Обучить токенизатор на новых данных

# Преобразование текстов в последовательности
new_sequences = tokenizer.texts_to_sequences(new_text_data)

# Дополнение последовательностей до одинаковой длины
pad_length = 200  # Должно совпадать с длиной, использованной при обучении
padded_new = pad_sequences(new_sequences, maxlen=pad_length)

# Применение модели к новым данным
predictions = model.predict(padded_new)
predicted_classes = np.argmax(predictions, axis=-1)

# Если хотите преобразовать предсказанные классы обратно в названия
predicted_job_titles = le.inverse_transform(predicted_classes)

# Добавление предсказаний в DataFrame для вывода
new_df['job_title'] = predicted_job_titles

data_res = new_df
# Сохранение результатов в новый файл
#output_path = '/home/jupyter/datasphere/project/output_file2.csv'  # Укажите путь к выходному файлу
#new_df.to_csv(output_path, index=False)
#print(f"Предсказания успешно сохранены в файл {output_path}")

In [ ]:
pip install joblib catboost

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
import joblib

def predict_salaries(input_csv_path, model_path, output_csv_path):
    # Загрузка обученной модели
    model = joblib.load(model_path)

    # Загрузка данных из CSV файла
    data = pd.read_csv(input_csv_path, low_memory=False)
    data = data.fillna("")

    # Отбор нужных столбцов
    data = data[['id', 'academic_degree', 'accommodation_capability', 'additional_requirements',
                 'busy_type', 'career_perspective', 'education', 'education_speciality', 
                 'is_mobility_program', 'need_medcard', 'other_vacancy_benefit', 
                 'position_requirements', 'position_responsibilities', 'regionName', 
                 'regionNameTerm', 'company_business_size', 'required_certificates', 
                 'required_drive_license', 'required_experience',  'schedule_type', 'professionalSphereName', 
                 'languageKnowledge', 'hardSkills', 'softSkills']]

    # Удаляем ненужные столбцы для предсказания
    useless_columns = ['id', 'salary']  # Убедитесь, что 'salary' удален, если он уже есть
    X = data.drop(columns=useless_columns, errors='ignore')

    # Определяем категориальные признаки
    categorical_features = [col for col in X.columns if X[col].dtype == 'object']

    # Обработка категориальных данных
    for feature in categorical_features:
        X[feature] = X[feature].astype(str).fillna('missing')

    # Преобразуем все числовые колонки в числовой формат, заменяя пропуски
    for col in X.columns:
        if col not in categorical_features:
            X[col] = pd.to_numeric(X[col], errors='coerce').fillna(-1)

    # Предсказание заработной платы
    y_pred = model.predict(X)

    # Добавляем предсказанные значения в новый столбец "salary"
    data['salary'] = y_pred

    # Сохранение обновленного набора данных в новый CSV файл
    #data.to_csv(output_csv_path, index=False)
    return data

data_sal = predict_salaries("TEST_SAL.csv", "catboost_model.joblib", "sample_sumbision.csv")

In [ ]:
# Приведение столбца 'salary' к целочисленному типу в df1
data_sal['salary'] = data_sal['salary'].astype(int)

# Добавление колонки task_type к каждому DataFrame
data_res['task_type'] = 'RES'
data_sal['task_type'] = 'SAL'

# Объединение DataFrames
submission = pd.concat([data_res, data_sal])

# Выбор нужных колонок и сохранение результата
result = submission[['id', 'job_title', 'task_type', 'salary']]
result.to_csv('submission/submission_fin.csv', index=False, encoding='utf-8')